Zajímalo nás, jestli se dají školy rozdělit na několik skupin a zjistit, jestli jsou nějaké zajímavé tendence nebo anomalie, které nevidíme, jestli je korelace mezi výsledky a zájmem o školu, výsledky a existencí školní zkoušky. Protože celkový počet uchazečů a zájem o konkrétní školu se v čase mění, stejně jako se čas od času mění kritéria přijetí pro danou školu zejména (zavedení nebo zrušení školní zkoušky), pracovali jsme  s parametrem “škola + rok”. 

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import plotly.io as pli
import os
if not os.path.exists("images"):
    os.mkdir("images")
import sklearn
import plotly.express as px
import datapane as dp
from sklearn.cluster import KMeans 
import warnings
warnings.simplefilter('ignore')

ImportError: cannot import name 'get_params' from 'sklearn.cluster' (c:\users\realk\appdata\local\programs\python\python37\lib\site-packages\sklearn\cluster\__init__.py)

In [2]:
data = pd.read_csv("./data/complet5.gz", compression = "gzip", encoding = 'utf8')

In [3]:
print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28214 entries, 0 to 28213
Data columns (total 60 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DESCRIPTION              28214 non-null  object 
 1   COMENT                   44 non-null     object 
 2   SCHOOLCOURSE_ID          28214 non-null  int64  
 3   YEAR                     28214 non-null  int64  
 4   MAX_SUM                  28214 non-null  float64
 5   MAX_CZECH                28214 non-null  int64  
 6   PROCENTA_CZECH           28214 non-null  float64
 7   MAX_MATH                 28214 non-null  int64  
 8   PROCENTA_MATH            28214 non-null  float64
 9   MAX_GRADES               28214 non-null  float64
 10  PROCENTA_GRADES          28214 non-null  float64
 11  MAX_SP                   2673 non-null   float64
 12  PROCENTA_SP              28214 non-null  float64
 13  MAX_VR                   5164 non-null   float64
 14  PROCENTA_VR           

In [5]:
data.head(10)

,DESCRIPTION,COMENT,SCHOOLCOURSE_ID,YEAR,MAX_SUM,MAX_CZECH,PROCENTA_CZECH,MAX_MATH,PROCENTA_MATH,MAX_GRADES,...,ADDRESS,CITY,PSC,KRAJ,LAT,LNG,COURSE_CODE,NAME,DURATION,COURSE_ID
0,nepřijat z důvodu kapacity / škola nemá stanov...,NaN,1981,2019,162.0,50,30.86,50,30.86,12.0,...,Mezi Školami 2475/29,Praha,15 800,Hlavní město Praha,50.052284,14.340408,79-41/K-81,osmileté gymnázium,8,81
1,nepřijat z důvodu kapacity / škola nemá stanov...,NaN,1981,2019,162.0,50,30.86,50,30.86,12.0,...,Mezi Školami 2475/29,Praha,15 800,Hlavní město Praha,50.052284,14.340408,79-41/K-81,osmileté gymnázium,8,81
2,nepřijat z důvodu kapacity / škola nemá stanov...,NaN,1981,2019,162.0,50,30.86,50,30.86,12.0,...,Mezi Školami 2475/29,Praha,15 800,Hlavní město Praha,50.052284,14.340408,79-41/K-81,osmileté gymnázium,8,81
3,nepřijat z důvodu kapacity / škola nemá stanov...,NaN,1981,2019,162.0,50,30.86,50,30.86,12.0,...,Mezi Školami 2475/29,Praha,15 800,Hlavní město Praha,50.052284,14.340408,79-41/K-81,osmileté gymnázium,8,81
4,nepřijat z důvodu kapacity / škola nemá stanov...,NaN,1981,2019,162.0,50,30.86,50,30.86,12.0,...,Mezi Školami 2475/29,Praha,15 800,Hlavní město Praha,50.052284,14.340408,79-41/K-81,osmileté gymnázium,8,81
5,nepřijat z důvodu kapacity / škola nemá stanov...,NaN,1981,2019,162.0,50,30.86,50,30.86,12.0,...,Mezi Školami 2475/29,Praha,15 800,Hlavní město Praha,50.052284,14.340408,79-41/K-81,osmileté gymnázium,8,81
6,nepřijat z důvodu kapacity / škola nemá stanov...,NaN,1981,2019,162.0,50,30.86,50,30.86,12.0,...,Mezi Školami 2475/29,Praha,15 800,Hlavní město Praha,50.052284,14.340408,79-41/K-81,osmileté gymnázium,8,81
7,nepřijat z důvodu kapacity / škola nemá stanov...,NaN,1981,2019,162.0,50,30.86,50,30.86,12.0,...,Mezi Školami 2475/29,Praha,15 800,Hlavní město Praha,50.052284,14.340408,79-41/K-81,osmileté gymnázium,8,81
8,nepřijat z důvodu kapacity / škola nemá stanov...,NaN,1981,2019,162.0,50,30.86,50,30.86,12.0,...,Mezi Školami 2475/29,Praha,15 800,Hlavní město Praha,50.052284,14.340408,79-41/K-81,osmileté gymnázium,8,81
9,nepřijat z důvodu kapacity / škola nemá stanov...,NaN,1981,2019,162.0,50,30.86,50,30.86,12.0,...,Mezi Školami 2475/29,Praha,15 800,Hlavní město Praha,50.052284,14.340408,79-41/K-81,osmileté gymnázium,8,81



##### Připravíme k agregaci sloupce, které budeme k analýtze potřebovat


In [6]:
data2 = data[["SCHOOLCOURSE_ID", "PROCENTA_MATH", "PROCENTA_GRADES", "PROCENTA_CZECH", "IS_SCHOOL_EXAM", "MIN_MATH_ADMITTED", "MIN_CZECH_ADMITTED", 
              "PROCENTA_VR", "PROCENTA_AJ", "PROCENTA_SP", "PROCENTA_OTHER", 
              "PROCENTA_CONTESTS", "IS_MIN_CRITERIA",
              "UCHAZECU_NA_JEDNO_MISTO", "ADMITTED_ID", "SCHOOL_NAME", "YEAR", "ADMITTED_PERCENT"]] 
data2["PROCENTA_MATH"]=data2["PROCENTA_MATH"].astype(float)
data2["PROCENTA_GRADES"]=data2["PROCENTA_GRADES"].astype(float)
data2["PROCENTA_CZECH"]=data2["PROCENTA_CZECH"].astype(float)
data2["PROCENTA_VR"]=data2["PROCENTA_VR"].astype(float)
data2["PROCENTA_AJ"]=data2["PROCENTA_AJ"].astype(float)
data2["PROCENTA_SP"]=data2["PROCENTA_SP"].astype(float)
data2["PROCENTA_OTHER"]=data2["PROCENTA_OTHER"].astype(float)
data2["PROCENTA_CONTESTS"]=data2["PROCENTA_CONTESTS"].astype(float)
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28214 entries, 0 to 28213
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   SCHOOLCOURSE_ID          28214 non-null  int64  
 1   PROCENTA_MATH            28214 non-null  float64
 2   PROCENTA_GRADES          28214 non-null  float64
 3   PROCENTA_CZECH           28214 non-null  float64
 4   IS_SCHOOL_EXAM           28214 non-null  bool   
 5   MIN_MATH_ADMITTED        26401 non-null  float64
 6   MIN_CZECH_ADMITTED       26401 non-null  float64
 7   PROCENTA_VR              28214 non-null  float64
 8   PROCENTA_AJ              28214 non-null  float64
 9   PROCENTA_SP              28214 non-null  float64
 10  PROCENTA_OTHER           28214 non-null  float64
 11  PROCENTA_CONTESTS        28214 non-null  float64
 12  IS_MIN_CRITERIA          28214 non-null  bool   
 13  UCHAZECU_NA_JEDNO_MISTO  28214 non-null  float64
 14  ADMITTED_ID           

#### Grupujeme podle školy a roku

In [7]:
dataset = data2.groupby(["YEAR", "SCHOOL_NAME"], as_index=False).agg({"ADMITTED_PERCENT":'max', 
                                                                      "SCHOOLCOURSE_ID":'max',
                                                        "IS_SCHOOL_EXAM": 'all', 
                                                        "PROCENTA_MATH":'max', 
                                                        "PROCENTA_GRADES":'max', 
                                                        "PROCENTA_CZECH":'max', 
                                                        "MIN_MATH_ADMITTED":'max',
                                                        "MIN_CZECH_ADMITTED":'max', 
                                                        "UCHAZECU_NA_JEDNO_MISTO":'max', 
                                                        "PROCENTA_VR":'max', 
                                                        "PROCENTA_SP":'max', 
                                                        "PROCENTA_AJ":'max',
                                                        "PROCENTA_OTHER":'max', 
                                                        "PROCENTA_CONTESTS":'max',
                                                        "IS_MIN_CRITERIA": 'all'
                                                                     })


In [8]:
dataset["IS_SCHOOL_EXAM"]=dataset["IS_SCHOOL_EXAM"].astype(float)

In [9]:
column_list = ["PROCENTA_MATH", "PROCENTA_CZECH"] 
dataset["PROCENTA_CERMAT"] = dataset[column_list].sum(axis=1)

In [10]:
column_list = ["MIN_MATH_ADMITTED", "MIN_CZECH_ADMITTED"] 
dataset["MIN_CERMAT"] = dataset[column_list].sum(axis=1)

In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90 entries, 0 to 89
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   YEAR                     90 non-null     int64  
 1   SCHOOL_NAME              90 non-null     object 
 2   ADMITTED_PERCENT         90 non-null     float64
 3   SCHOOLCOURSE_ID          90 non-null     int64  
 4   IS_SCHOOL_EXAM           90 non-null     float64
 5   PROCENTA_MATH            90 non-null     float64
 6   PROCENTA_GRADES          90 non-null     float64
 7   PROCENTA_CZECH           90 non-null     float64
 8   MIN_MATH_ADMITTED        84 non-null     float64
 9   MIN_CZECH_ADMITTED       84 non-null     float64
 10  UCHAZECU_NA_JEDNO_MISTO  90 non-null     float64
 11  PROCENTA_VR              90 non-null     float64
 12  PROCENTA_SP              90 non-null     float64
 13  PROCENTA_AJ              90 non-null     float64
 14  PROCENTA_OTHER           90 

In [12]:
dataset

,YEAR,SCHOOL_NAME,ADMITTED_PERCENT,SCHOOLCOURSE_ID,IS_SCHOOL_EXAM,PROCENTA_MATH,PROCENTA_GRADES,PROCENTA_CZECH,MIN_MATH_ADMITTED,MIN_CZECH_ADMITTED,UCHAZECU_NA_JEDNO_MISTO,PROCENTA_VR,PROCENTA_SP,PROCENTA_AJ,PROCENTA_OTHER,PROCENTA_CONTESTS,IS_MIN_CRITERIA,PROCENTA_CERMAT,MIN_CERMAT
0,2017,Gymnázium Budějovická,13.23,1481,0.0,41.32,9.09,41.32,37.0,36.0,7.56,0.0,0.0,0.00,0.00,8.26,False,82.64,73.0
1,2017,Gymnázium Chodovická,44.88,2181,0.0,45.45,4.55,45.45,NaN,NaN,2.23,0.0,0.0,0.00,0.00,4.55,False,90.90,0.0
2,2017,Gymnázium Elišky Krásnohorské,13.06,481,0.0,42.37,10.17,42.37,35.0,32.0,7.66,0.0,0.0,0.00,0.00,5.08,True,84.74,67.0
3,2017,Gymnázium Hlivická,19.24,1581,0.0,39.37,15.75,39.37,38.0,34.0,5.20,0.0,0.0,0.00,0.00,5.51,False,78.74,72.0
4,2017,Gymnázium J.G.Jarkovského,22.69,1781,0.0,43.48,6.52,43.48,32.0,33.0,4.41,0.0,0.0,0.00,0.00,6.52,False,86.96,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2020,Gymnázium a Hudební škola hl.m.Prahy ZUŠ,63.16,1681,1.0,29.94,4.79,29.94,11.0,24.0,1.58,0.0,0.0,0.00,35.33,0.00,False,59.88,35.0
86,2020,Gymnázium prof. Jana Patočky,14.46,181,1.0,41.67,8.33,41.67,37.0,34.0,6.91,0.0,0.0,8.33,0.00,0.00,True,83.34,71.0
87,2020,Gymnázium Českolipská,17.80,881,0.0,47.62,4.76,47.62,33.0,36.0,5.62,0.0,0.0,0.00,0.00,0.00,True,95.24,69.0
88,2020,Gymnázium Špitálská,12.95,1081,0.0,43.90,9.76,43.90,40.0,38.0,7.72,0.0,0.0,0.00,0.00,2.44,False,87.80,78.0


In [13]:
dataset.isnull().sum(axis=1)

0     0
1     2
2     0
3     0
4     0
     ..
85    0
86    0
87    0
88    0
89    0
Length: 90, dtype: int64

Vylučujeme z dataframe, nad kterým jsme děláme analyzu, dvě školy. Jedná se o Chodovickou, u které nejsou k dispozici data výsledků zkoušek, a Gymnázium a Hudební škola, která je velmi specifická. Má velkou váhu školní zkoušky, ale malý zájem a malou úspěšnost. Je to dáno tím, že se jedná o školu pro hudebně nadané děti. Její chování ovlivňuje celkový výsledek a zároveň nijak nepomáhá rodičům “běžných” dětí. 

In [117]:
#Chodovická
dataset = dataset.dropna()

In [118]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 0 to 89
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   YEAR                     84 non-null     int64  
 1   SCHOOL_NAME              84 non-null     object 
 2   ADMITTED_PERCENT         84 non-null     float64
 3   SCHOOLCOURSE_ID          84 non-null     int64  
 4   IS_SCHOOL_EXAM           84 non-null     float64
 5   PROCENTA_MATH            84 non-null     float64
 6   PROCENTA_GRADES          84 non-null     float64
 7   PROCENTA_CZECH           84 non-null     float64
 8   MIN_MATH_ADMITTED        84 non-null     float64
 9   MIN_CZECH_ADMITTED       84 non-null     float64
 10  UCHAZECU_NA_JEDNO_MISTO  84 non-null     float64
 11  PROCENTA_VR              84 non-null     float64
 12  PROCENTA_SP              84 non-null     float64
 13  PROCENTA_AJ              84 non-null     float64
 14  PROCENTA_OTHER           84 

In [119]:
#vytvoreni datasetu bez hudebni skoly
is_hudebni = dataset["SCHOOLCOURSE_ID"] != 1681
dataset_no_hudebni = dataset[is_hudebni] 

In [120]:

dataset_no_hudebni.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 0 to 89
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   YEAR                     80 non-null     int64  
 1   SCHOOL_NAME              80 non-null     object 
 2   ADMITTED_PERCENT         80 non-null     float64
 3   SCHOOLCOURSE_ID          80 non-null     int64  
 4   IS_SCHOOL_EXAM           80 non-null     float64
 5   PROCENTA_MATH            80 non-null     float64
 6   PROCENTA_GRADES          80 non-null     float64
 7   PROCENTA_CZECH           80 non-null     float64
 8   MIN_MATH_ADMITTED        80 non-null     float64
 9   MIN_CZECH_ADMITTED       80 non-null     float64
 10  UCHAZECU_NA_JEDNO_MISTO  80 non-null     float64
 11  PROCENTA_VR              80 non-null     float64
 12  PROCENTA_SP              80 non-null     float64
 13  PROCENTA_AJ              80 non-null     float64
 14  PROCENTA_OTHER           80 

# Analýza

### Proměnná pro rok, se kterým chceme pracovat:

In [121]:
year = "2019"

#### 4 parametry, 3 clustery

Shlukujeme školy podle váhy jednotné zkoušky CERMAT v celkovém hodnocení dané školy, minimálního počtu bodů za CERMAT, školní zkoušky (je nebo není) a váhy známek. Chceme potvrdit domněnku, zda opravdu existují stabilní skupiny škol, v rámci kterých bude šance dostat se u uchazečů se stejnými předpoklady (úspěšnost v CERMATu, známky, připravenost k školní zkoušce) také stejná. Nejdřív zkusíme 3 skupiny. 


In [122]:
kmeans = KMeans(n_clusters=3)
clusters43 = kmeans.fit_predict(dataset_no_hudebni[["MIN_CERMAT", "PROCENTA_CERMAT", 
                                                    "IS_SCHOOL_EXAM", "PROCENTA_GRADES"
                                                   ]])
clusters43

array([1, 1, 1, 1, 2, 1, 0, 0, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 2, 2, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 2, 2, 0, 1, 2, 2, 2, 2, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 0, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1])

In [123]:
dataset_no_hudebni.loc[:, "clusters43"]=clusters43.astype(str)
dataset_no_hudebni.loc[:,"IS_SCHOOL_EXAM"] = dataset_no_hudebni.loc[:, "IS_SCHOOL_EXAM"].replace(0, 0.5)

In [124]:
fig43 = px.scatter(
    dataset_no_hudebni, x='PROCENTA_CERMAT', y='MIN_CERMAT', color = "clusters43",
    size='IS_SCHOOL_EXAM', hover_data=["SCHOOL_NAME", "YEAR"])
fig43.show()


In [125]:
fig43y = px.scatter(
    dataset_no_hudebni.query(f"YEAR=={year}"), 
    x='PROCENTA_CERMAT', y='MIN_CERMAT', color="clusters43",
    size='IS_SCHOOL_EXAM', hover_data=["SCHOOL_NAME", "YEAR"])
fig43y.show()

Nelibi se nam, ze jsou v jednom clusteru (1) skoro všechny školy se školní zkouškou a menší váhou CERMATu bez ohledu na minimální počet bodů nutný k přijetí. A chceme vudět, jestli ovlivní výsledek skutečnost, kolik dětí se na školu hlásí. 

### 4 parametry, 4 clustery

Shlukujeme tedy školy podle váhy jednotné zkoušky CERMAT v celkovém hodnocení dané školy, počtu uchazečů na jedno místo, minimálního počtu bodů za CERMAT a váhy známek.



In [126]:
kmeans = KMeans(n_clusters=4)
clusters54 = kmeans.fit_predict(dataset_no_hudebni[["PROCENTA_CERMAT", "UCHAZECU_NA_JEDNO_MISTO",  
                                                     "MIN_CERMAT",  "PROCENTA_GRADES"]])
dataset_no_hudebni["clusters54"]=clusters54.astype(str)

In [127]:
clusters54

array([0, 0, 0, 0, 3, 0, 1, 1, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 3, 2, 1, 1, 0, 2, 2, 2, 2, 1, 2, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 3, 2, 1, 0, 2, 2, 2, 2, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       3, 2, 1, 0, 2, 3, 2, 2, 0, 3, 0, 0, 0, 0])

In [128]:
##!!!!!!!!!! SKOLNI ZKOUSKA JAKO STRING ANO/NE


fig54 = px.scatter(dataset_no_hudebni, x='PROCENTA_CERMAT', y='MIN_CERMAT', color="clusters54",
                size='IS_SCHOOL_EXAM', 
                trendline="ols",
                hover_data=["SCHOOL_NAME", "YEAR"], 
                labels={"YEAR":"Rok",
                        "SCHOOL_NAME": "Škola", 
                        "clusters54": "Skupina",
                        "IS_SCHOOL_EXAM":"Školní zkouška",  
                        "PROCENTA_CERMAT": "Váha zkoušky CERMAT", 
                        "MIN_CERMAT":"Počet bodů CERMAT u posledního přijatého"
                       }
                  )

In [116]:
fig54.update_layout(
    title="Rozdělení škol na skupiny",
    
    xaxis_title="Procentuální váha jednotné zkoušky CERMAT (čeština + matematika)",
    yaxis_title="Počet bodů za jednotnou zkoušku u posledního přijatého",
    legend_title="Skupiny")

#### To je mnohem lepší!

### Popis skupin


### Modré

Nejpočetnější skupina, která zahrnuje školy bez školní zkoušky s průměrnou až nadprůměrnou
váhou CERMATu a průměrnými až nadprůměrnými výsledky jednotné zkoušky. "Nejhorší" výsledky se vztahují k 
r. 2018, ve kterém výsledky CERMATu byly obecně slabší. 

Zajímavé je, že Gymnázium prof. Jaroslava Patočky, které školní 
zkoušku sice koná,  v letech 2017 a 2020 mělo podobné výsledky jako členy této skupiny.

#### Fialové

Tvoří ji školy, které mají školní zkoušku a proto váha CERMATu je logicky spíš nízká. Na rozdíl od clusteru 3 mají 
menší (průměrný až podprůměrný) minimální počet bodů za CERMAT, který nekoreluje s váhou zkoušky CERMAT.

#### Zelené

Tato skupina škol má průměrnou váhu zkoušky CERMAT (zohledňuje více známky za předchozí vzdělávání) 
a zároveň - až na 1 výjimku (Gymnázium Jaroslava Patočky v letech 2018 a 2019) - nepořádá školní zkoušku. 
K tomu, aby se student na tuto školu dostal, stačí průměrný až podprůměrný (50 až 67) počet bodů za CERMAT

#### Červené

Jedná se o školy, kam není vůnec snadno se dostat. Mají velkou váhu školní zkoušky 
a zároveň je třeba mít opravdu nadprůměrné výsledky ze zkoušky jednotné.  Tvoří ji Gymnázium Jana Keplera. V roce 2020, kdy byly výsledky CERMATu obecně vyšší, se sem dostaly další dvě školy, i když i tak jsou od Keplerky poměrně daleko.  

## Publikace na Datapane

#Datapane
dp.Report(
    dp.Plot(fig54)).publish(name='Shlukování škol s trendlines', open=True)

### ------------------------------------

In [105]:
dataset_no_hudebni.loc[:, "IS_SCHOOL_EXAM"] = dataset_no_hudebni.loc[:,"IS_SCHOOL_EXAM"].replace("0.5", "NE")
dataset_no_hudebni.loc[:, "IS_SCHOOL_EXAM"] = dataset_no_hudebni.loc[:,"IS_SCHOOL_EXAM"].replace('1.0', "ANO")

# Korelace

### Zájem o školu a váha Cermatu, publikace na Datapane pro blog

In [106]:
dataset_no_hudebni["IS_SCHOOL_EXAM"]= dataset_no_hudebni["IS_SCHOOL_EXAM"].astype(str) 
kor_c_a= px.scatter(dataset_no_hudebni
                    #.query(f'YEAR=={year}')
                    , x='PROCENTA_CERMAT', y="ADMITTED_PERCENT", 
                hover_data=['SCHOOL_NAME', 'IS_SCHOOL_EXAM', 'YEAR'], 
                 trendline="ols",  
                 color="IS_SCHOOL_EXAM",
                 
                labels={"YEAR":"Rok","SCHOOL_NAME": "Škola",
                      "IS_SCHOOL_EXAM":"Školní zkouška",  
                        "PROCENTA_CERMAT": "Váha zkoušky CERMAT, %", 
                       "ADMITTED_PERCENT":"přijatých z celkového počtu uchazečů, %"}
                   )

In [107]:
kor_c_a.update_layout(
    title="Zájem o školu vs váha zkoušky Cermat",
    
    xaxis_title="Procentuální váha jednotné zkoušky CERMAT (čeština + matematika)",
    yaxis_title="% přijatých z celkového počtu uchazečů",
   
    
    legend_title="Školní zkouška")

#Publikace na Datapane
dp.Report(
    dp.Plot(kor_c_a)).publish(name='Zájem o školy a jednotná zkouška - podle clusterů', open=True)

### Komentar 
Mírná kolelace - čím méně zkoušek, tím větší zájem - tedy tím menší procento přijatých. 
Patočka je jasná volba, pokud nechcete školu s velkou vánou Cermatu anuu s celkou váhou školní zkoušky ani s velkým
počtem uchazečů. Špitálská je jasná volba, pokud tam chcete, jinak můžete zkusit velmi podobnou vyhlášenou Nad Alejí

### Zájem vs. známky

In [109]:
dataset_no_hudebni["IS_SCHOOL_EXAM"]= dataset_no_hudebni["IS_SCHOOL_EXAM"].astype(str) 
fig3 = px.scatter(dataset_no_hudebni
                  #.query(f'YEAR=={year}')
                  , x='PROCENTA_GRADES', y="UCHAZECU_NA_JEDNO_MISTO", 
                hover_data=['SCHOOL_NAME', "YEAR"], 
                 trendline="ols", 
                 color="IS_SCHOOL_EXAM"
                 # ,color="clusters54"
                 )
fig3.show()


#### Minimální úspěšnost vs známky

In [110]:
kor_c_g = px.scatter(dataset_no_hudebni
                  #.query(f'YEAR=={year}')
                  , x="PROCENTA_GRADES", y='MIN_CERMAT', 
                hover_data=['SCHOOL_NAME'], 
                 trendline="ols",
                #color="IS_SCHOOL_EXAM", 
                 color="clusters54")
kor_c_g.show()

#Publikace na Datapane
dp.Report(
    dp.Plot(kor_c_g)).publish(name='Známky a minimální úspšnost - podle clusterů', open=True)

#### Minimální úspěšnost vs. zájem

In [111]:
fig3 = px.scatter(dataset_no_hudebni
                  #.query(f'YEAR=={year}')
                  , x="UCHAZECU_NA_JEDNO_MISTO", y='MIN_CERMAT', 
                hover_data=['SCHOOL_NAME', 'YEAR'], 
                 trendline="ols",  
                color="IS_SCHOOL_EXAM", 
                 #color="clusters54"
                 )
fig3.show()

#### Minimální úspěšnost vs váha Cermatu

In [112]:
fig5 = px.scatter(dataset_no_hudebni
                  #.query(f'YEAR=={year}')
                  , 
                  x="PROCENTA_CERMAT", y='MIN_CERMAT', 
                hover_data=['SCHOOL_NAME', "YEAR"], 
                 trendline="ols",  
                 #color="IS_SCHOOL_EXAM"
                  
                 color="clusters54"
                 )
fig5.show()